In [ ]:
# instalação do Weights & Biases
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
# instalação do pytest
!pip install pytest pytest-sugar

In [ ]:
# importação das bibliotecas
import wandb
import numpy as np
import pandas as pd

In [ ]:
# wandb login
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# criando um arquivo de test
%%file test_data.py

import pytest
import wandb
import pandas as pd
import numpy as np
import os
from PIL import Image

# inicializando uma execução do wandb para verificar o pré-processamento
run = wandb.init(entity="flamigos", project="cnn_animation", job_type="data_check")

@pytest.fixture(scope="session")
def data():
    artifact = run.use_artifact('flamigos/cnn_animation/label_data:latest', type='clean_data').file()
    artifact2 = run.use_artifact('flamigos/cnn_animation/img_data:latest', type='clean_data').file()
    labels = pd.read_csv(artifact)
    data_img = np.load(artifact2)

    return labels, data_img


def test_data_length(data):
    # verificamos se temos a quantidade necessária de dados e se os dois artefatos têm o mesmo tamanho
    labels, data_img = data
    assert (len(labels) > 8800) and (len(data_img) > 8800) and (len(labels) == len(data_img))


def test_image_data_shape(data):
    # verifica se todos os valores do artefato data_img têm o mesmo formato
    labels, data_img = data
    for i in data_img:
      assert i.shape == data_img[0].shape


def test_label_data_types(data):
    # verifica se todos os valores do artefato de rótulos são do mesmo tipo
    labels, data_img = data
    for i in range(len(labels["animation Type"])):
      assert type(labels["animation Type"][i]) == type(labels["animation Type"][0])


def test_img_data_types(data):
    # verifica se todos os valores do artefato data_img têm o mesmo tipo
    labels, data_img = data
    for i in range(len(data_img)):
      assert type(data_img[i]) == type(data_img[0])


def test_image_data_valid(data):
    # verifica se os dados de img_dat são uma imagem válida
    # com este teste não precisamos de uma verificação nula no img_data!
    labels, data_img = data
    for i in data_img:
        img = i*255
        img = img.astype(np.uint8)
        image = Image.fromarray(img)
        assert (image.size == (128, 128) and image.mode == 'RGBA')


def test_label_range(data):
    # testa se os valores da categoria estão dentro de um intervalo de valores esperado
    labels, data_img = data
    assert labels["animation Type"].isin([0, 1]).all()


def test_null_values_from_labels(data):
    # testa se há dados faltantes no quadro de dados
    labels, data_img = data
    assert labels.isnull().any().any() == False

    # finaliza a run
    run.finish()

Writing test_data.py


In [ ]:
# inicializa o teste
!pytest . -vv

Test session starts (platform: linux, Python 3.10.12, pytest 7.4.3, pytest-sugar 0.9.7)
cachedir: .pytest_cache
rootdir: /content
plugins: sugar-0.9.7, anyio-3.7.1
collected 7 items                                                                                  

 test_data.py::test_data_length ✓                                                     14% █▌        
 test_data.py::test_image_data_shape ✓                                                29% ██▉       
 test_data.py::test_label_data_types ✓                                                43% ████▍     
 test_data.py::test_img_data_types ✓                                                  57% █████▊    
 test_data.py::test_image_data_valid ✓                                                71% ███████▎  
 test_data.py::test_label_range ✓                                                     86% ████████▋ 
 test_data.py::test_null_values_from_labels ✓                                        100% ██████████

Results (77.84s (0:01:17)):